In [ ]:
# import packages
from scipy.optimize import minimize, fmin, fminbound, fmin_cg   # finding optimal parameters in models
from scipy import stats                     # statistical tools
import os                                   # operating system tools
import numpy as np                          # matrix/array functions
import random
import pandas as pd                         # loading and manipulating data
import ipywidgets as widgets                # interactive display
import matplotlib.pyplot as plt             # plotting
from matplotlib.pyplot import figure
import requests
import warnings

# warnings.filterwarnings("error")
warnings.filterwarnings("ignore")

In [ ]:
df_full = pd.read_csv("RL.csv")


Important terms-
1. Rescorla Wagner- learning occurs when there is a prediction error.
2. V(t) is investment at time t.
3. r(t) is reward at time t. 
4. α is the learning rate, which is bounded between 0 and 1. It captures the extent to which the prediction error δ, which equals [r(t)−V(t)], updates the value (i.e., a higher α value will put greater weight on the δ). 
5. Assume that the initial value, V(0)=0, although it is possible to treat the V(0) as a free parameter of the model (this is also the intercept)
6. ζ is the 'inverse temperature' parameter that controls the level of stochasticity in the choice, ranging from ζ=0 for completely random responding and ζ=∞ for deterministically choosing the highest value option. This means that someone with a large ζ would almost always choose the option with the highest value (V(t)), while someone with a small ζ would explore other options more frequently. [Note that this means we expect choosing the largest investment to be the most non-random strategy]

Therefore, we will have two free parameters - 
1. The learning rate, α 
2. The inverse temperature ζ.

# Minimize the negative loglikelihood

Now minimize the negative loglikelihood. For the minimization algorithm - 
1. Loop through possible combinations of α and ζ for subject to minimize the negative loglikelihood.
2. Each subject (i) will have a seperate set of parameters αi and ζi , which respectively correspond to the learning rate and inverse temperature 


In [ ]:
def softmax(q_values, beta):
    # Inputs:
    #       q_values: value of different actions
    #       beta: inverse temperature, also known as exploitation parameter
    # Outputs:
    #       choice_probs: model's predicted probability of actions

    # Numerator represents value of utility of single choice
    numerator = np.exp(np.multiply(q_values, beta))

    # Denominator represents sum of total value of utilities
    denominator = np.sum(np.exp(np.multiply(q_values, beta)))
    
    # Outputs
    choice_probs = numerator / denominator
        
    return choice_probs
    
### Temporal difference model
# Negative log likelihood and Rescorla Wagner function

def NegLL(params):
    
    global invest, reward, k
    
    alpha, beta = params
    
    # Set seed for reproducibility
    np.random.seed(random.randint(1,1000))                         
    epsilon = .00000000001;
    
    # number of trials
    time = len(invest) 
        
    # iniQ is the initial probability for all invest amounts possible for each trial
    # iniQ is set at 0.5 for the first trial, and updates every trial based on the amount chosen in previous trial
    iniQ = [0.5]*k
    
    # create empty array to save probabilities for choices made
    choiceprob = []
    
    for t in range(time): # iterate through rounds
        
        # compute probabilities for k=11 
        # choosing to invest from 0-10
        # use the softmax rule
        p = softmax(np.array(iniQ), beta)
        
        # select probability for actual decision 
        choiceprob.append(p[invest[t]])
        
        # reward at time t
        r_t = (10 - invest[t]) + reward[t] # leftover + return

        # reward at time t
        # r_t = reward[t] # return
        
        # prediction error
        # delta = r_t - Q_t
        # ALTERNATIVE FROM CHANG AND ASSOCIATES
        delta = r_t - iniQ[invest[t]] # compute pe = reward - q_value[choice]

        # learning
        a_d = alpha * delta
        a_d = round(a_d, 7)
        
        # save probability calculated in this round so we only use one prior round's probabilities each time
        # update value to reflect prediction error
        # make sure current value is 0.5
        # iniQ = [0.5]*k
        iniQ[invest[t]] = iniQ[invest[t]] + a_d
        
        # print(f"Updated probability for {invest[t]}: {iniQ[invest[t]]}")
        
    probability_choicesmade = []
    
    for choice in choiceprob:
        probability_choicesmade.append(max(choice, epsilon))
    
    NegLL = -np.sum(np.log(probability_choicesmade)) # this is the negative log likelihood for all rounds played across all games for this subject
        
    return NegLL



# Analyze full dataset by Agency x Status x Generosity for hyper parameters

In [ ]:
status = df_full.status_contrast.unique()
agency = df_full.agency_contrast.unique()
generosity = df_full.generosity_contrast1.unique()

df = df_full[['MoneyTrusted', 'PartnerReturn', 'generosity_contrast1']]

# Use different guess ranges of alpha and zeta to see if you get the same results

optimized = {}
k=11

for a in agency:
    if a == 0.5:
        key = "human"
    elif a == -0.5:
        key = "AI"
    df_a = df[df.agency_contrast == a]
    for s in status:
        if s == 0.5:
            key = f"{key}_highstatus"
        elif s == -0.5:
            key = f"{key}_lowstatus"
        df_s = df_a[df_a.status_contrast == s]
        for g in generosity:
            if g == 0.5:
                key = f"{key}_generous"
            elif g == 0:
                key = f"{key}_neutral"
            elif g == -0.5:
                key = f"{key}_greedy"
            df_g = df_s[df_s.generosity_contrast1 == g]

            print(f"Estimating for: {key}")

            invest = df_g['MoneyTrusted'].tolist()
            reward = df_g['PartnerReturn'].tolist()

            optimized_alpha_list = []
            optimized_zeta_list = []
            function_value_list = []
            n_iterations_list = []
            n_funcevals_list = []
            success_list = []
            message_list = []

            if key not in optimized.keys():
                optimized[key] = {}

            for alpha in np.linspace(0,1,3):
                for zeta in np.linspace(0,100,3):

                    init_guess = np.array([alpha, zeta])

                    # Specify boundaries
                    bounds = ((0, 1), (0, None))

                    # FMINSEARCH = method 'Nelder-Mead'
                    optimized_result = minimize(NegLL, init_guess, 
                                                method = 'Nelder-Mead', bounds = bounds, options = {'maxiter':1000})

                    # Go through result dict
                    optimized_alpha = optimized_result['x'][0]
                    optimized_zeta = optimized_result['x'][1]
                    function_value = optimized_result['fun']
                    n_iterations = optimized_result['nit']
                    n_funcevals = optimized_result['nfev']
                    success = optimized_result['success']
                    message = optimized_result['message']

                    optimized_alpha_list.append(optimized_alpha)
                    optimized_zeta_list.append(optimized_zeta)
                    function_value_list.append(function_value)
                    n_iterations_list.append(n_iterations)
                    n_funcevals_list.append(n_funcevals)
                    success_list.append(success)
                    message_list.append(message)


            # get the minimum value in the function_value list
            min_value = min(function_value_list)

            # return the index of minimum value 
            min_index = function_value_list.index(min_value)

            # subset all lists by min_index
            optimized_alpha_list = optimized_alpha_list[min_index]
            optimized_zeta_list = optimized_zeta_list[min_index]
            function_value_list = function_value_list[min_index]
            n_iterations_list = n_iterations_list[min_index]
            n_funcevals_list = n_funcevals_list[min_index]
            success_list = success_list[min_index]
            message_list = message_list[min_index]

            # Add result dict to final dict
            optimized[key]['optimized_alpha'] = optimized_alpha_list
            optimized[key]['optimized_zeta'] = optimized_zeta_list
            optimized[key]['function_value'] = function_value_list
            optimized[key]['n_iterations'] = n_iterations_list
            optimized[key]['n_funcevals'] = n_funcevals_list
            optimized[key]['success'] = success_list
            optimized[key]['message'] = message_list # optimized params

optimized_df = pd.DataFrame.from_dict(optimized, orient='index')
optimized_df.reset_index(inplace=True)
optimized_df.columns = ['Generosity level', 'optimized_alpha', 'optimized_zeta', 'function_value', 
                        'n_iterations', 'n_funcevals', 'success', 'message']

optimized_df.head()


In [ ]:
optimized_df.to_csv('optimized_hyperparameters_.csv', index=False)


# Analyze per subject

In [ ]:
# # Create loop to loop through and fminsearch

# subjects = df_full.Subject.unique()

# k = 11
# iniQ=[0.5]*k

# # create dictionary to save each subject's data for each game and round
# subjects_optimized = {}

# for subject in subjects:

#     print(f"Subject: {subject}") 
#     optimized_alpha_list = []
#     optimized_zeta_list = []
#     function_value_list = []
#     n_iterations_list = []
#     n_funcevals_list = []
#     success_list = []
#     message_list = []

#     if subject not in subjects_optimized.keys():
#         subjects_optimized[subject] = {}

#     df = df_full[df_full.Subject == subject]
    
#     # START
#     invest = df["MoneyTrusted"].tolist()
#     reward = df["PartnerReturn"].tolist()

#     np.random.seed(random.randint(1,1000)) # set seed for reproducibility

#     # guess several different starting points for alpha
#     # using np.linspace to evenly space the numbers 
#     for alpha in np.linspace(0,1,100):  # Read through papers to get an idea of the usual range
#         for zeta in np.linspace(0,10,100): # Read through papers to get an idea of the usual range

#             # guesses for alpha, zeta will change on each loop
#             init_guess = np.array([alpha, zeta])

#             # Specify boundaries
#             bounds = ((0, 1), (0, None))

#             # FMINSEARCH = method 'Nelder-Mead'
#             optimized_result = minimize(NegLL, init_guess, 
#                                         method = 'Nelder-Mead', bounds = bounds)

#             # Go through result dict and select optimized params
#             optimized_alpha = optimized_result['x'][0]
#             optimized_zeta = optimized_result['x'][1]
#             function_value = optimized_result['fun']
#             n_iterations = optimized_result['nit']
#             n_funcevals = optimized_result['nfev']
#             success = optimized_result['success']
#             message = optimized_result['message']
            
#             # Add results to subject-level lists
#             optimized_alpha_list.append(optimized_alpha)
#             optimized_zeta_list.append(optimized_zeta)
#             function_value_list.append(function_value)
#             n_iterations_list.append(n_iterations)
#             n_funcevals_list.append(n_funcevals)
#             success_list.append(success)
#             message_list.append(message)
            
#     # get the minimum value in the function_value list
#     min_value = min(function_value_list)
 
#     # return the index of minimum value 
#     min_index = function_value_list.index(min_value)
    
#     # subset all lists by min_index
#     optimized_alpha_list = optimized_alpha_list[min_index]
#     optimized_zeta_list = optimized_zeta_list[min_index]
#     function_value_list = function_value_list[min_index]
#     n_iterations_list = n_iterations_list[min_index]
#     n_funcevals_list = n_funcevals_list[min_index]
#     success_list = success_list[min_index]
#     message_list = message_list[min_index]
            
#     # Add final results to subject_optimized_df
#     if 'optimized_alpha' not in subjects_optimized[subject].keys():
#         subjects_optimized[subject]['optimized_alpha'] =  optimized_alpha_list
#     if 'optimized_zeta' not in subjects_optimized[subject].keys():
#         subjects_optimized[subject]['optimized_zeta'] =  optimized_zeta_list
#     if 'function_value' not in subjects_optimized[subject].keys():
#         subjects_optimized[subject]['function_value'] =  function_value_list
#     if 'n_iterations' not in subjects_optimized[subject].keys():
#         subjects_optimized[subject]['n_iterations'] =  n_iterations_list
#     if 'n_funcevals' not in subjects_optimized[subject].keys():
#         subjects_optimized[subject]['n_funcevals'] =  n_funcevals_list
#     if 'success' not in subjects_optimized[subject].keys():
#         subjects_optimized[subject]['success'] =  success_list
#     if 'message' not in subjects_optimized[subject].keys():
#         subjects_optimized[subject]['message'] =  message_list

# optimized_df = pd.DataFrame.from_dict(subjects_optimized, orient='index')
# optimized_df.reset_index(inplace=True)


In [ ]:
# optimized_df.columns = ['Subject', 'optimized_alpha', 'optimized_zeta', 
#                                    'function_value', 'n_iterations', 'n_funcevals', 'success', 'message']
# optimized_df.head()


In [ ]:
# optimized_df.to_csv('optimized_subj_.csv', index=False)

## Analyze by condition per subject: Generosity

In [ ]:
# # Create loop to loop through and fminsearch

# subjects = df_full.Subject.unique()
# generosity = df_full.generosity_contrast1.unique()

# k = 11
# iniQ=[0.5]*k

# # create dictionary to save each subject's data for each game and round
# # subjects_optimized = {}

# for subject in subjects:
#     df = df_full[df_full.Subject == subject]
#     print(f"Subject: {subject}") 
    
#     if subject not in subjects_optimized.keys():
#         subjects_optimized[subject] = {}
            
#     for g in generosity:
#         df_g = df[df.generosity_contrast1 == g]
        
#         if g == 0.5:
#             key = "generous"
#         elif g == 0:
#             key = "neutral"
#         elif g == -0.5:
#             key = "greedy"
#         print(f"Estimating for: {key}")
        
#         # create lists to save values
#         optimized_alpha_list = []
#         optimized_zeta_list = []
#         function_value_list = []
#         n_iterations_list = []
#         n_funcevals_list = []
#         success_list = []
#         message_list = []

#         if key not in subjects_optimized[subject].keys():
#             subjects_optimized[subject][key] = {}
    
#         # START
#         invest = df_g["MoneyTrusted"].tolist()
#         reward = df_g["PartnerReturn"].tolist()

#         np.random.seed(random.randint(1,1000)) # set seed for reproducibility

#         # guess several different starting points for alpha
#         # using np.linspace to evenly space the numbers 
#         for alpha in np.linspace(0,1,100):  # Read through papers to get an idea of the usual range
#             for zeta in np.linspace(0,10,100): # Read through papers to get an idea of the usual range

#                 # guesses for alpha, zeta will change on each loop
#                 init_guess = np.array([alpha, zeta])

#                 # Specify boundaries
#                 bounds = ((0, 1), (0, None))

#                 # FMINSEARCH = method 'Nelder-Mead'
#                 optimized_result = minimize(NegLL, init_guess, 
#                                             method = 'Nelder-Mead', bounds = bounds)

#                 # Go through result dict and select optimized paramsc
#                 optimized_alpha = optimized_result['x'][0]
#                 optimized_zeta = optimized_result['x'][1] 
#                 function_value = optimized_result['fun']
#                 n_iterations = optimized_result['nit']
#                 n_funcevals = optimized_result['nfev']
#                 success = optimized_result['success']
#                 message = optimized_result['message']

#                 # Add results to subject-level lists
#                 optimized_alpha_list.append(optimized_alpha)
#                 optimized_zeta_list.append(optimized_zeta)
#                 function_value_list.append(function_value)
#                 n_iterations_list.append(n_iterations)
#                 n_funcevals_list.append(n_funcevals)
#                 success_list.append(success)
#                 message_list.append(message)

#         # get the minimum value in the function_value list
#         min_value = min(function_value_list)

#         # return the index of minimum value 
#         min_index = function_value_list.index(min_value)

#         # subset all lists by min_index
#         optimized_alpha_list = optimized_alpha_list[min_index]
#         optimized_zeta_list = optimized_zeta_list[min_index]
#         function_value_list = function_value_list[min_index]
#         n_iterations_list = n_iterations_list[min_index]
#         n_funcevals_list = n_funcevals_list[min_index]
#         success_list = success_list[min_index]
#         message_list = message_list[min_index]

#         # Add final results to subject_optimized_df
#         if 'optimized_alpha' not in subjects_optimized[subject][key].keys():
#             subjects_optimized[subject][key]['optimized_alpha'] =  optimized_alpha_list
#         if 'optimized_zeta' not in subjects_optimized[subject][key].keys():
#             subjects_optimized[subject][key]['optimized_zeta'] =  optimized_zeta_list
#         if 'function_value' not in subjects_optimized[subject][key].keys():
#             subjects_optimized[subject][key]['function_value'] =  function_value_list
#         if 'n_iterations' not in subjects_optimized[subject][key].keys():
#             subjects_optimized[subject][key]['n_iterations'] =  n_iterations_list
#         if 'n_funcevals' not in subjects_optimized[subject][key].keys():
#             subjects_optimized[subject][key]['n_funcevals'] =  n_funcevals_list
#         if 'success' not in subjects_optimized[subject][key].keys():
#             subjects_optimized[subject][key]['success'] =  success_list
#         if 'message' not in subjects_optimized[subject][key].keys():
#             subjects_optimized[subject][key]['message'] =  message_list

# optimized_df = pd.DataFrame.from_dict({(i,j): subjects_optimized[i][j] 
#                                        for i in subjects_optimized.keys() 
#                                        for j in subjects_optimized[i].keys()},
#                                       orient='index')

# optimized_df.reset_index(inplace=True)


In [ ]:
# optimized_df.columns = ['Subject', 'Generosity', 'optimized_alpha', 'optimized_zeta', 'function_value', 'n_iterations', 'n_funcevals', 
#                         'success', 'message']
# optimized_df.head()


In [ ]:
# optimized_df.to_csv('optimized_subj_bygenerosity_.csv', index=False)


## Analyze by condition per subject: Status

In [ ]:
# # Create loop to loop through and fminsearch

# subjects = df_full.Subject.unique()
# status = df_full.status_contrast.unique()

# k = 11
# iniQ=[0.5]*k

# # create dictionary to save each subject's data for each game and round
# subjects_optimized = {}

# for subject in subjects[68:]:
#     df = df_full[df_full.Subject == subject]
#     print(f"Subject: {subject}") 
    
#     if subject not in subjects_optimized.keys():
#         subjects_optimized[subject] = {}
            
#     for s in status:
#         df_s = df[df.status_contrast == s]
        
#         if s == 0.5:
#             key = "high status"
#         elif s == -0.5:
#             key = "low status"
            
#         print(f"Estimating for: {key}")
        
#         # create lists to save values
#         optimized_alpha_list = []
#         optimized_zeta_list = []
#         function_value_list = []
#         n_iterations_list = []
#         n_funcevals_list = []
#         success_list = []
#         message_list = []

#         if key not in subjects_optimized[subject].keys():
#             subjects_optimized[subject][key] = {}
    
#         # START
#         invest = df_s["MoneyTrusted"].tolist()
#         reward = df_s["PartnerReturn"].tolist()

#         np.random.seed(random.randint(1,1000)) # set seed for reproducibility

#         # guess several different starting points for alpha
#         # using np.linspace to evenly space the numbers 
#         for alpha in np.linspace(0,1,100):  # Read through papers to get an idea of the usual range
#             for zeta in np.linspace(0,10,100): # Read through papers to get an idea of the usual range

#                 # guesses for alpha, zeta will change on each loop
#                 init_guess = np.array([alpha, zeta])

#                 # Specify boundaries
#                 bounds = ((0, 1), (0, None))

#                 # FMINSEARCH = method 'Nelder-Mead'
#                 optimized_result = minimize(NegLL, init_guess, 
#                                             method = 'Nelder-Mead', bounds = bounds)

#                 # Go through result dict and select optimized paramsc
#                 optimized_alpha = optimized_result['x'][0]
#                 optimized_zeta = optimized_result['x'][1] 
#                 function_value = optimized_result['fun']
#                 n_iterations = optimized_result['nit']
#                 n_funcevals = optimized_result['nfev']
#                 success = optimized_result['success']
#                 message = optimized_result['message']

#                 # Add results to subject-level lists
#                 optimized_alpha_list.append(optimized_alpha)
#                 optimized_zeta_list.append(optimized_zeta)
#                 function_value_list.append(function_value)
#                 n_iterations_list.append(n_iterations)
#                 n_funcevals_list.append(n_funcevals)
#                 success_list.append(success)
#                 message_list.append(message)

#         # get the minimum value in the function_value list
#         min_value = min(function_value_list)

#         # return the index of minimum value 
#         min_index = function_value_list.index(min_value)

#         # subset all lists by min_index
#         optimized_alpha_list = optimized_alpha_list[min_index]
#         optimized_zeta_list = optimized_zeta_list[min_index]
#         function_value_list = function_value_list[min_index]
#         n_iterations_list = n_iterations_list[min_index]
#         n_funcevals_list = n_funcevals_list[min_index]
#         success_list = success_list[min_index]
#         message_list = message_list[min_index]

#         # Add final results to subject_optimized_df
#         if 'optimized_alpha' not in subjects_optimized[subject][key].keys():
#             subjects_optimized[subject][key]['optimized_alpha'] =  optimized_alpha_list
#         if 'optimized_zeta' not in subjects_optimized[subject][key].keys():
#             subjects_optimized[subject][key]['optimized_zeta'] =  optimized_zeta_list
#         if 'function_value' not in subjects_optimized[subject][key].keys():
#             subjects_optimized[subject][key]['function_value'] =  function_value_list
#         if 'n_iterations' not in subjects_optimized[subject][key].keys():
#             subjects_optimized[subject][key]['n_iterations'] =  n_iterations_list
#         if 'n_funcevals' not in subjects_optimized[subject][key].keys():
#             subjects_optimized[subject][key]['n_funcevals'] =  n_funcevals_list
#         if 'success' not in subjects_optimized[subject][key].keys():
#             subjects_optimized[subject][key]['success'] =  success_list
#         if 'message' not in subjects_optimized[subject][key].keys():
#             subjects_optimized[subject][key]['message'] =  message_list

# optimized_df = pd.DataFrame.from_dict({(i,j): subjects_optimized[i][j] 
#                                        for i in subjects_optimized.keys() 
#                                        for j in subjects_optimized[i].keys()},
#                                       orient='index')

# optimized_df.reset_index(inplace=True)


In [ ]:
# optimized_df.columns = ['Subject', 'Status', 'optimized_alpha', 'optimized_zeta', 'function_value', 
#                         'n_iterations', 'n_funcevals', 'success', 'message']

# optimized_df.head()


In [ ]:
# optimized_df.to_csv('optimized_subj_bystatus_.csv', index=False)


## Analyze by condition per subject: Agency x Status x Generosity

In [ ]:
# Create loop to loop through and fminsearch

subjects = df_full.Subject.unique()
status = df_full.status_contrast.unique()
agency = df_full.agency_contrast.unique()
generosity = df_full.generosity_contrast1.unique()

k = 11
iniQ=[0.5]*k

# create dictionary to save each subject's data for each game and round
subjects_optimized = {}

for subject in subjects[250:]:
    df = df_full[df_full.Subject == subject]
    print(f"Subject: {subject}") 
    
    if subject not in subjects_optimized.keys():
        subjects_optimized[subject] = {}
    
    for a in agency:
        df_a = df[df.agency_contrast == a]
        
        if len(df_a.index) > 0:
            if a == 0.5:
                key1 = "Human"
            elif a == -0.5:
                key1 = "AI"

            for s in status:
                df_s = df_a[df_a.status_contrast == s]

                if s == 0.5:
                    key2 = "High status"
                elif s == -0.5:
                    key2 = "Low status"

                for g in generosity:
                    df_g = df_s[df_s.generosity_contrast1 == g]

                    if g == 0.5:
                        key3 = "Generous"
                    elif g == 0:
                        key3 = "Neutral"
                    elif g == -0.5:
                        key3 = "Greedy"

                    print(f"Estimating for: {key1}_{key2}_{key3}")

                    # create lists to save values
                    optimized_alpha_list = []
                    optimized_zeta_list = []
                    function_value_list = []
                    n_iterations_list = []
                    n_funcevals_list = []
                    success_list = []
                    message_list = []

                    if key1 not in subjects_optimized[subject].keys():
                        subjects_optimized[subject][key1] = {}
                    if key2 not in subjects_optimized[subject][key1].keys():
                        subjects_optimized[subject][key1][key2] = {}
                    if key3 not in subjects_optimized[subject][key1][key2].keys():
                        subjects_optimized[subject][key1][key2][key3] = {}

                    # START
                    invest = df_s["MoneyTrusted"].tolist()
                    reward = df_s["PartnerReturn"].tolist()

                    np.random.seed(random.randint(1,1000)) # set seed for reproducibility

                    # guess several different starting points for alpha
                    # using np.linspace to evenly space the numbers 
                    for alpha in np.linspace(0,1,100):  # Read through papers to get an idea of the usual range
                        for zeta in np.linspace(0,10,100): # Read through papers to get an idea of the usual range

                            # guesses for alpha, zeta will change on each loop
                            init_guess = np.array([alpha, zeta])

                            # Specify boundaries
                            bounds = ((0, 1), (0, None))

                            # FMINSEARCH = method 'Nelder-Mead'
                            optimized_result = minimize(NegLL, init_guess, 
                                                        method = 'Nelder-Mead', bounds = bounds)

                            # Go through result dict and select optimized paramsc
                            optimized_alpha = optimized_result['x'][0]
                            optimized_zeta = optimized_result['x'][1] 
                            function_value = optimized_result['fun']
                            n_iterations = optimized_result['nit']
                            n_funcevals = optimized_result['nfev']
                            success = optimized_result['success']
                            message = optimized_result['message']

                            # Add results to subject-level lists
                            optimized_alpha_list.append(optimized_alpha)
                            optimized_zeta_list.append(optimized_zeta)
                            function_value_list.append(function_value)
                            n_iterations_list.append(n_iterations)
                            n_funcevals_list.append(n_funcevals)
                            success_list.append(success)
                            message_list.append(message)

                    # get the minimum value in the function_value list
                    min_value = min(function_value_list)

                    # return the index of minimum value 
                    min_index = function_value_list.index(min_value)

                    # subset all lists by min_index
                    optimized_alpha_list = optimized_alpha_list[min_index]
                    optimized_zeta_list = optimized_zeta_list[min_index]
                    function_value_list = function_value_list[min_index]
                    n_iterations_list = n_iterations_list[min_index]
                    n_funcevals_list = n_funcevals_list[min_index]
                    success_list = success_list[min_index]
                    message_list = message_list[min_index]

                    # Add final results to subject_optimized_df
                    if 'optimized_alpha' not in subjects_optimized[subject][key1][key2][key3].keys():
                        subjects_optimized[subject][key1][key2][key3]['optimized_alpha'] =  optimized_alpha_list
                    if 'optimized_zeta' not in subjects_optimized[subject][key1][key2][key3].keys():
                        subjects_optimized[subject][key1][key2][key3]['optimized_zeta'] =  optimized_zeta_list
                    if 'function_value' not in subjects_optimized[subject][key1][key2][key3].keys():
                        subjects_optimized[subject][key1][key2][key3]['function_value'] =  function_value_list
                    if 'n_iterations' not in subjects_optimized[subject][key1][key2][key3].keys():
                        subjects_optimized[subject][key1][key2][key3]['n_iterations'] =  n_iterations_list
                    if 'n_funcevals' not in subjects_optimized[subject][key1][key2][key3].keys():
                        subjects_optimized[subject][key1][key2][key3]['n_funcevals'] =  n_funcevals_list
                    if 'success' not in subjects_optimized[subject][key1][key2][key3].keys():
                        subjects_optimized[subject][key1][key2][key3]['success'] =  success_list
                    if 'message' not in subjects_optimized[subject][key1][key2][key3].keys():
                        subjects_optimized[subject][key1][key2][key3]['message'] =  message_list


#### FMINCON

In [ ]:
# COMPARE COMPARE COMPARE
# FMINSEARCH AGAINST FMIN UNCONSTRAINED, GRADIENT DESCENT
# SOMETIMES OPTIMIZATION FUNCTION MAY BE THE ISSUE
# MAKE SURE RESULTS ARE ONLY A COUPLE OF POINTS DIFFERENT FROM EACH OTHER
# 2.05 AND 2.07 ARE SAME BUT 2 AND 3 ARE NOT
# MIGHT MEAN ONE OPTIMIZATION PROCEDURE IS LANDING ON A LOCAL INSTEAD OF GLOBAL MINIMA

# Gradient descent papers!!! - test one optimization algorithm against another

### For next time: use one subject's data to compare MATLAB to Python, compare changes across different ranges
### Fit this function on actual data - week after next
### Make sure function is customizable to the other decision-rules - after that


# optimized_df = pd.DataFrame.from_dict({(i,j,k,l): subjects_optimized[i][j][k][l] 
#                                        for i in subjects_optimized.keys() 
#                                        for j in subjects_optimized[i].keys()
#                                        for k in subjects_optimized[i][j].keys() 
#                                        for l in subjects_optimized[i][j][k].keys()},
#                                       orient='index')

# optimized_df.reset_index(inplace=True)
# optimized_df.columns = ['Subject', 'Agency', 'Status', 'Generosity', 'optimized_alpha', 'optimized_zeta', 
#                         'function_value', 'n_iterations', 'n_funcevals', 'success', 'message']
# optimized_df.head()

# previous_df = pd.read_csv('optimized_subj_byagencystatusgenerosity_.csv')
# previous_df.head()

# optimized_df = pd.concat([previous_df, optimized_df])
# optimized_df.head()
# optimized_df.Subject.unique()

# optimized_df.to_csv('optimized_subj_byagencystatusgenerosity_.csv', index=False)


In [ ]:
# missing = []
# for subject in subjects:
#     if subject not in optimized_df.Subject.unique().tolist():
#         missing.append(subject)
        
# missing


In [ ]:
### Tested function against same function in MATLAB script

invest = [3, 6, 9]
reward = [5, 0, 3]
alpha = 1
zeta = 1
k = 11

NegLL([alpha, zeta])

### Answers same



In [ ]:
I_+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++